In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Data

## Import

In [ ]:
import pandas as pd
import re
import spacy
from spacy.tokenizer import Tokenizer
from nltk.stem import PorterStemmer

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
!python -m spacy download en_core_web_sm

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-09 18:44:01.181059: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 7.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# CLEANED
df_cleaned = pd.read_csv("/content/drive/MyDrive/NLP PROJECT/TODA/TODA_cleand.csv").iloc[:,2:]
# # CLEANED
df_typo = pd.read_csv("/content/drive/MyDrive/NLP PROJECT/TODA/TODA_typo.csv").iloc[:,1:]

In [ ]:
df_cleaned = df_cleaned[df_cleaned['cleaned'].apply(len)<1000].reset_index(drop=True)[['text','cleaned','class']]
df_typo = df_typo[['text','cleaned','jspell','class']]

In [ ]:
df_cleaned.shape,df_typo.shape

((24783, 3), (24783, 4))

In [ ]:
pd.concat([df_cleaned.head(),df_typo.head()])

,text,cleaned,class,jspell
0,!!! RT @mayasolovely: As a woman you shouldn't...,!!! RT @mayasolovely As a woman you shouldn't ...,0,NaN
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,!!!!! RT @mleew17 boy dats coldtyga dwn bad fo...,1,NaN
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1,NaN
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,!!!!!!!!! RT @CGAnderson @vivabased she look l...,1,NaN
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,!!!!!!!!!!!!! RT @ShenikaRoberts The shit you ...,1,NaN
0,!!! RT @mayasolovely: As a woman you shouldn't...,!!! RT @mayasolovely As a woman you shouldn't ...,0,!!! RT @mayasolovely As a woman you shouldn't ...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,!!!!! RT @mleew17 boy dats coldtyga dwn bad fo...,1,!!!!! RT @level17 boy days coldtyga down bad f...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1,!!!!!!! RT @UrKindOfBrand Dawn!!!! RT @80baby4...
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,!!!!!!!!! RT @CGAnderson @vivabased she look l...,1,!!!!!!!!! RT @ANDerson @vivabased she look lik...
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,!!!!!!!!!!!!! RT @ShenikaRoberts The shit you ...,1,!!!!!!!!!!!!! RT @ShenikaRoberts The shit you ...


## Preprocess

In [ ]:
from sklearn.utils import shuffle
df_cleaned = shuffle(df_cleaned,random_state=42).reset_index(drop=True)
df_typo = shuffle(df_typo,random_state=42).reset_index(drop=True)

In [ ]:
pd.concat([df_cleaned.head(),df_typo.head()])

,text,cleaned,class,jspell
0,934 8616\ni got a missed call from yo bitch,934 8616i got a missed call from yo bitch,1,NaN
1,RT @KINGTUNCHI_: Fucking with a bad bitch you ...,RT @KINGTUNCHI Fucking with a bad bitch you go...,1,NaN
2,RT @eanahS__: @1inkkofrosess lol my credit ain...,RT @eanahS @1inkkofrosess lol my credit ain't ...,0,NaN
3,RT @Maxin_Betha Wipe the cum out of them faggo...,RT @MaxinBetha Wipe the cum out of them faggot...,1,NaN
4,Niggas cheat on they bitch and don't expect no...,Niggas cheat on they bitch and don't expect no...,1,NaN
0,934 8616\ni got a missed call from yo bitch,934 8616i got a missed call from yo bitch,1,934 8616i got a missed call from yo bitch
1,RT @KINGTUNCHI_: Fucking with a bad bitch you ...,RT @KINGTUNCHI Fucking with a bad bitch you go...,1,RT @KINGTUNCHI Fucking with a bad bitch you go...
2,RT @eanahS__: @1inkkofrosess lol my credit ain...,RT @eanahS @1inkkofrosess lol my credit ain't ...,0,RT @means @1inkkofrosess lol my credit ain't n...
3,RT @Maxin_Betha Wipe the cum out of them faggo...,RT @MaxinBetha Wipe the cum out of them faggot...,1,RT @MaxinBetha Wipe the cut out of them faggot...
4,Niggas cheat on they bitch and don't expect no...,Niggas cheat on they bitch and don't expect no...,1,Niggas cheat on they bitch and don't expect to...


In [ ]:
from sklearn.utils import shuffle

def preprocess(text):

  text = re.sub('http\S+', '', text)

  text = re.sub('@\S+', '', text)

  delete_e = re.compile("[^a-zA-Z0-9\!\?\$\%\' ]")
  text = delete_e.sub("",text)
  text = text.lower()
  return text

In [ ]:
df_cleaned['cleaned'] = df_cleaned['cleaned'].apply(preprocess)
df_typo['cleaned'] = df_typo['cleaned'].apply(preprocess)
df_typo['jspell'] = df_typo['jspell'].apply(preprocess)

In [ ]:
pd.concat([df_cleaned.head(),df_typo.head()])

,text,cleaned,class,jspell
0,934 8616\ni got a missed call from yo bitch,934 8616i got a missed call from yo bitch,1,NaN
1,RT @KINGTUNCHI_: Fucking with a bad bitch you ...,rt fucking with a bad bitch you gone need som...,1,NaN
2,RT @eanahS__: @1inkkofrosess lol my credit ain...,rt lol my credit ain't no where near good b...,0,NaN
3,RT @Maxin_Betha Wipe the cum out of them faggo...,rt wipe the cum out of them faggot rt contac...,1,NaN
4,Niggas cheat on they bitch and don't expect no...,niggas cheat on they bitch and don't expect no...,1,NaN
0,934 8616\ni got a missed call from yo bitch,934 8616i got a missed call from yo bitch,1,934 8616i got a missed call from yo bitch
1,RT @KINGTUNCHI_: Fucking with a bad bitch you ...,rt fucking with a bad bitch you gone need som...,1,rt fucking with a bad bitch you gone need som...
2,RT @eanahS__: @1inkkofrosess lol my credit ain...,rt lol my credit ain't no where near good b...,0,rt lol my credit ain't no where near good b...
3,RT @Maxin_Betha Wipe the cum out of them faggo...,rt wipe the cum out of them faggot rt contac...,1,rt wipe the cut out of them faggot rt contac...
4,Niggas cheat on they bitch and don't expect no...,niggas cheat on they bitch and don't expect no...,1,niggas cheat on they bitch and don't expect to...


## Tokenizing

In [ ]:
nlp = spacy.load('en_core_web_sm')
tokenizer = Tokenizer(nlp.vocab)

df_cleaned['Tokens'] = df_cleaned['cleaned'].apply(lambda x: [ token.text for token in tokenizer(x)])
df_typo['Tokens'] = df_typo['jspell'].apply(lambda x: [ token.text for token in tokenizer(x)])

In [ ]:
pd.concat([df_cleaned.head(),df_typo.head()])

,text,cleaned,class,Tokens,jspell
0,934 8616\ni got a missed call from yo bitch,934 8616i got a missed call from yo bitch,1,"[934, 8616i, got, a, missed, call, from, yo, b...",NaN
1,RT @KINGTUNCHI_: Fucking with a bad bitch you ...,rt fucking with a bad bitch you gone need som...,1,"[rt, , fucking, with, a, bad, bitch, you, gon...",NaN
2,RT @eanahS__: @1inkkofrosess lol my credit ain...,rt lol my credit ain't no where near good b...,0,"[rt, , lol, my, credit, ain't, no, where, ne...",NaN
3,RT @Maxin_Betha Wipe the cum out of them faggo...,rt wipe the cum out of them faggot rt contac...,1,"[rt, , wipe, the, cum, out, of, them, faggot,...",NaN
4,Niggas cheat on they bitch and don't expect no...,niggas cheat on they bitch and don't expect no...,1,"[niggas, cheat, on, they, bitch, and, don't, e...",NaN
0,934 8616\ni got a missed call from yo bitch,934 8616i got a missed call from yo bitch,1,"[934, 8616i, got, a, missed, call, from, yo, b...",934 8616i got a missed call from yo bitch
1,RT @KINGTUNCHI_: Fucking with a bad bitch you ...,rt fucking with a bad bitch you gone need som...,1,"[rt, , fucking, with, a, bad, bitch, you, gon...",rt fucking with a bad bitch you gone need som...
2,RT @eanahS__: @1inkkofrosess lol my credit ain...,rt lol my credit ain't no where near good b...,0,"[rt, , lol, my, credit, ain't, no, where, ne...",rt lol my credit ain't no where near good b...
3,RT @Maxin_Betha Wipe the cum out of them faggo...,rt wipe the cum out of them faggot rt contac...,1,"[rt, , wipe, the, cut, out, of, them, faggot,...",rt wipe the cut out of them faggot rt contac...
4,Niggas cheat on they bitch and don't expect no...,niggas cheat on they bitch and don't expect no...,1,"[niggas, cheat, on, they, bitch, and, don't, e...",niggas cheat on they bitch and don't expect to...


# STOPWORD

In [ ]:
STOP_WORDS = nlp.Defaults.stop_words

# CLEANED DATA
i=0
tokens = []
for doc in nlp.pipe(df_cleaned['cleaned']):
  doc_tokens = []
  for token in doc:
    if (token.text not in STOP_WORDS)&(token.is_punct==False)&(token.is_space==False):
      doc_tokens.append(token.text)
  i+=1
  if i%100000==0:print(i)
  tokens.append(doc_tokens)
df_cleaned['Not Stopwords'] = tokens
print('Cleaned Data Tokenized!!!')

# TYPO DATA
i=0
tokens = []
for doc in nlp.pipe(df_typo['jspell']):
  doc_tokens = []
  for token in doc:
    if (token.text not in STOP_WORDS)&(token.is_punct==False)&(token.is_space==False):
      doc_tokens.append(token.text)
  i+=1
  if i%100000==0:print(i)
  tokens.append(doc_tokens)
df_typo['Not Stopwords'] = tokens
print('Typo Data Tokenized!!!')

Cleaned Data Tokenized!!!
Typo Data Tokenized!!!


In [ ]:
pd.concat([df_cleaned.head(),df_typo.head()])

,text,cleaned,class,Tokens,Not Stopwords,jspell
0,934 8616\ni got a missed call from yo bitch,934 8616i got a missed call from yo bitch,1,"[934, 8616i, got, a, missed, call, from, yo, b...","[934, 8616i, got, missed, yo, bitch]",NaN
1,RT @KINGTUNCHI_: Fucking with a bad bitch you ...,rt fucking with a bad bitch you gone need som...,1,"[rt, , fucking, with, a, bad, bitch, you, gon...","[rt, fucking, bad, bitch, gone, need, money, l...",NaN
2,RT @eanahS__: @1inkkofrosess lol my credit ain...,rt lol my credit ain't no where near good b...,0,"[rt, , lol, my, credit, ain't, no, where, ne...","[rt, lol, credit, ai, near, good, know, right,...",NaN
3,RT @Maxin_Betha Wipe the cum out of them faggo...,rt wipe the cum out of them faggot rt contac...,1,"[rt, , wipe, the, cum, out, of, them, faggot,...","[rt, wipe, cum, faggot, rt, contact, lens, wil...",NaN
4,Niggas cheat on they bitch and don't expect no...,niggas cheat on they bitch and don't expect no...,1,"[niggas, cheat, on, they, bitch, and, don't, e...","[niggas, cheat, bitch, expect, pay, whatsoever...",NaN
0,934 8616\ni got a missed call from yo bitch,934 8616i got a missed call from yo bitch,1,"[934, 8616i, got, a, missed, call, from, yo, b...","[934, 8616i, got, missed, yo, bitch]",934 8616i got a missed call from yo bitch
1,RT @KINGTUNCHI_: Fucking with a bad bitch you ...,rt fucking with a bad bitch you gone need som...,1,"[rt, , fucking, with, a, bad, bitch, you, gon...","[rt, fucking, bad, bitch, gone, need, money, l...",rt fucking with a bad bitch you gone need som...
2,RT @eanahS__: @1inkkofrosess lol my credit ain...,rt lol my credit ain't no where near good b...,0,"[rt, , lol, my, credit, ain't, no, where, ne...","[rt, lol, credit, ai, near, good, know, right,...",rt lol my credit ain't no where near good b...
3,RT @Maxin_Betha Wipe the cum out of them faggo...,rt wipe the cum out of them faggot rt contac...,1,"[rt, , wipe, the, cut, out, of, them, faggot,...","[rt, wipe, cut, faggot, rt, contact, lens, wil...",rt wipe the cut out of them faggot rt contac...
4,Niggas cheat on they bitch and don't expect no...,niggas cheat on they bitch and don't expect no...,1,"[niggas, cheat, on, they, bitch, and, don't, e...","[niggas, cheat, bitch, expect, pay, whatsoever...",niggas cheat on they bitch and don't expect to...


# STEMMING

In [ ]:
ps = PorterStemmer()

# CLEANED DATA
i=0
tokens = []
for doc in df_cleaned['Not Stopwords']:
  doc_tokens = []
  for token in doc:
    try:
      doc_tokens.append(ps.stem(token))
    except:
      print(token)
      doc_tokens.append(token)
  tokens.append(doc_tokens)
  i+=1
  if i%100000==0:print(i)
df_cleaned['stems'] = tokens
print('Cleaned Data Stemmed!!!')

# TYPO DATA
i=0
tokens = []
for doc in df_typo['Not Stopwords']:
  doc_tokens = []
  for token in doc:
    try:
      doc_tokens.append(ps.stem(token))
    except:
      print(token)
      doc_tokens.append(token)
  tokens.append(doc_tokens)
  i+=1
  if i%100000==0:print(i)
df_typo['stems'] = tokens
print('Typo Data Stemmed!!!')

Cleaned Data Stemmed!!!
Typo Data Stemmed!!!


In [ ]:
pd.concat([df_cleaned.head(),df_typo.head()])

,text,cleaned,class,Tokens,Not Stopwords,stems,jspell
0,934 8616\ni got a missed call from yo bitch,934 8616i got a missed call from yo bitch,1,"[934, 8616i, got, a, missed, call, from, yo, b...","[934, 8616i, got, missed, yo, bitch]","[934, 8616i, got, miss, yo, bitch]",NaN
1,RT @KINGTUNCHI_: Fucking with a bad bitch you ...,rt fucking with a bad bitch you gone need som...,1,"[rt, , fucking, with, a, bad, bitch, you, gon...","[rt, fucking, bad, bitch, gone, need, money, l...","[rt, fuck, bad, bitch, gone, need, money, lil,...",NaN
2,RT @eanahS__: @1inkkofrosess lol my credit ain...,rt lol my credit ain't no where near good b...,0,"[rt, , lol, my, credit, ain't, no, where, ne...","[rt, lol, credit, ai, near, good, know, right,...","[rt, lol, credit, ai, near, good, know, right,...",NaN
3,RT @Maxin_Betha Wipe the cum out of them faggo...,rt wipe the cum out of them faggot rt contac...,1,"[rt, , wipe, the, cum, out, of, them, faggot,...","[rt, wipe, cum, faggot, rt, contact, lens, wil...","[rt, wipe, cum, faggot, rt, contact, len, wild...",NaN
4,Niggas cheat on they bitch and don't expect no...,niggas cheat on they bitch and don't expect no...,1,"[niggas, cheat, on, they, bitch, and, don't, e...","[niggas, cheat, bitch, expect, pay, whatsoever...","[nigga, cheat, bitch, expect, pay, whatsoev, y...",NaN
0,934 8616\ni got a missed call from yo bitch,934 8616i got a missed call from yo bitch,1,"[934, 8616i, got, a, missed, call, from, yo, b...","[934, 8616i, got, missed, yo, bitch]","[934, 8616i, got, miss, yo, bitch]",934 8616i got a missed call from yo bitch
1,RT @KINGTUNCHI_: Fucking with a bad bitch you ...,rt fucking with a bad bitch you gone need som...,1,"[rt, , fucking, with, a, bad, bitch, you, gon...","[rt, fucking, bad, bitch, gone, need, money, l...","[rt, fuck, bad, bitch, gone, need, money, lil,...",rt fucking with a bad bitch you gone need som...
2,RT @eanahS__: @1inkkofrosess lol my credit ain...,rt lol my credit ain't no where near good b...,0,"[rt, , lol, my, credit, ain't, no, where, ne...","[rt, lol, credit, ai, near, good, know, right,...","[rt, lol, credit, ai, near, good, know, right,...",rt lol my credit ain't no where near good b...
3,RT @Maxin_Betha Wipe the cum out of them faggo...,rt wipe the cum out of them faggot rt contac...,1,"[rt, , wipe, the, cut, out, of, them, faggot,...","[rt, wipe, cut, faggot, rt, contact, lens, wil...","[rt, wipe, cut, faggot, rt, contact, len, wild...",rt wipe the cut out of them faggot rt contac...
4,Niggas cheat on they bitch and don't expect no...,niggas cheat on they bitch and don't expect no...,1,"[niggas, cheat, on, they, bitch, and, don't, e...","[niggas, cheat, bitch, expect, pay, whatsoever...","[nigga, cheat, bitch, expect, pay, whatsoev, y...",niggas cheat on they bitch and don't expect to...


# SAVE DATA

In [ ]:
# CLEANED DATA
df_cleaned_s = df_cleaned[['text','cleaned','Tokens','Not Stopwords','stems','class']]
df_cleaned_s.columns = ['text_raw','text_cleaned','tokens_raw','tokens_stop','tokens_stems','class']
# TYPO DATA
df_typo_s = df_typo[['text','cleaned','jspell','Tokens','Not Stopwords','stems','class']]
df_typo_s.columns = ['text_raw','text_cleaned','typo_corrected','tokens_raw','tokens_stop','tokens_stems','class']

In [ ]:
pd.concat([df_typo_s.head(),df_cleaned_s.head()])

,text_raw,text_cleaned,typo_corrected,tokens_raw,tokens_stop,tokens_stems,class
0,934 8616\ni got a missed call from yo bitch,934 8616i got a missed call from yo bitch,934 8616i got a missed call from yo bitch,"[934, 8616i, got, a, missed, call, from, yo, b...","[934, 8616i, got, missed, yo, bitch]","[934, 8616i, got, miss, yo, bitch]",1
1,RT @KINGTUNCHI_: Fucking with a bad bitch you ...,rt fucking with a bad bitch you gone need som...,rt fucking with a bad bitch you gone need som...,"[rt, , fucking, with, a, bad, bitch, you, gon...","[rt, fucking, bad, bitch, gone, need, money, l...","[rt, fuck, bad, bitch, gone, need, money, lil,...",1
2,RT @eanahS__: @1inkkofrosess lol my credit ain...,rt lol my credit ain't no where near good b...,rt lol my credit ain't no where near good b...,"[rt, , lol, my, credit, ain't, no, where, ne...","[rt, lol, credit, ai, near, good, know, right,...","[rt, lol, credit, ai, near, good, know, right,...",0
3,RT @Maxin_Betha Wipe the cum out of them faggo...,rt wipe the cum out of them faggot rt contac...,rt wipe the cut out of them faggot rt contac...,"[rt, , wipe, the, cut, out, of, them, faggot,...","[rt, wipe, cut, faggot, rt, contact, lens, wil...","[rt, wipe, cut, faggot, rt, contact, len, wild...",1
4,Niggas cheat on they bitch and don't expect no...,niggas cheat on they bitch and don't expect no...,niggas cheat on they bitch and don't expect to...,"[niggas, cheat, on, they, bitch, and, don't, e...","[niggas, cheat, bitch, expect, pay, whatsoever...","[nigga, cheat, bitch, expect, pay, whatsoev, y...",1
0,934 8616\ni got a missed call from yo bitch,934 8616i got a missed call from yo bitch,NaN,"[934, 8616i, got, a, missed, call, from, yo, b...","[934, 8616i, got, missed, yo, bitch]","[934, 8616i, got, miss, yo, bitch]",1
1,RT @KINGTUNCHI_: Fucking with a bad bitch you ...,rt fucking with a bad bitch you gone need som...,NaN,"[rt, , fucking, with, a, bad, bitch, you, gon...","[rt, fucking, bad, bitch, gone, need, money, l...","[rt, fuck, bad, bitch, gone, need, money, lil,...",1
2,RT @eanahS__: @1inkkofrosess lol my credit ain...,rt lol my credit ain't no where near good b...,NaN,"[rt, , lol, my, credit, ain't, no, where, ne...","[rt, lol, credit, ai, near, good, know, right,...","[rt, lol, credit, ai, near, good, know, right,...",0
3,RT @Maxin_Betha Wipe the cum out of them faggo...,rt wipe the cum out of them faggot rt contac...,NaN,"[rt, , wipe, the, cum, out, of, them, faggot,...","[rt, wipe, cum, faggot, rt, contact, lens, wil...","[rt, wipe, cum, faggot, rt, contact, len, wild...",1
4,Niggas cheat on they bitch and don't expect no...,niggas cheat on they bitch and don't expect no...,NaN,"[niggas, cheat, on, they, bitch, and, don't, e...","[niggas, cheat, bitch, expect, pay, whatsoever...","[nigga, cheat, bitch, expect, pay, whatsoev, y...",1


In [ ]:
df_cleaned_s.to_csv("/content/drive/MyDrive/NLP PROJECT/TODA(CLEANED)/BERT/TODA_CLEANED_01.csv")
df_typo_s.to_csv("/content/drive/MyDrive/NLP PROJECT/TODA(CLEANED)/BERT/TODA_TYPO_01.csv")